Kristupas Norvaiša

Imports

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image

Load the model ResNet50

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = models.resnet50(pretrained = True).to(device)
model.eval()

Download and preprocess the data

In [ ]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git
!pip3 install -r /content/OIDv4_ToolKit/requirements.txt
!python /content/OIDv4_ToolKit/main.py downloader --classes Goose Kite Magpie --type_csv train --limit 500
data_loader = data_loader('/content/OID/Dataset/train', batch_size = 32)

fatal: destination path 'OIDv4_ToolKit' already exists and is not an empty directory.
ERROR: Operation cancelled by user

		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Goose.


Open the classes labels file

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
with open('/content/drive/MyDrive/imagenet_classes.txt', 'r') as fid:
  all_labels = [ln.strip() for ln in fid]

Data loader

In [ ]:
def data_loader(data_dir, batch_size, shuffle = False):
    transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

    dataset = torchvision.datasets.ImageFolder('/content/OID/Dataset/train', transform = transform)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle = shuffle)
    return data_loader

Initialize empty lists for storing predicted classes and their probabilities

In [ ]:
predicted_classes = []
predicted_probs = []

Initialize empty lists for storing predicted and actual class ids

In [ ]:
predicted = []
predicted_words = []
actual = []

Create, iterate over the categories and initialize empty lists for storing predicted classes and their probabilities. # 99 goose, 21 kite, 18 magpie.

In [ ]:
categories = [99,21,18]  
threshold = 0.7;
for i in range(len(categories)):
  predicted_classes.append([])
  predicted_probs.append([])

Define a function to convert the predicted probability into a binary label

In [ ]:
def convert_by_threshold_for_class(probs, id):
  if(probs[id].item() > threshold):
    return 1
  return 0

Define a function to get the probability of a predicted class

In [ ]:
def convert_by_threshold_for_probs(probs, id):
  return probs[id].item()

Define a function to calculate the predicted classes and their probabilities for each image in the data loader

In [ ]:
def calculate(output):
    output_id = torch.argmax(output)
    output_id = output_id.item()
    predicted.append(output_id)
    predicted_words.append(all_labels[output_id])

    probs = F.softmax(output, dim=0)
    for i, category in enumerate(categories):
        predicted_add_m = predicted_classes[i]
        predicted_add_p = predicted_probs[i]
        predicted_add_m.append(convert_by_threshold_for_class(probs, category, threshold))
        predicted_add_p.append(probs[category].item())

Define a function to convert the actual class labels into a binary matrix

In [ ]:
def convert_actual_to_matrix(class_id, array):
    return [1 if i == class_id else 0 for i in array]

Initialize variables for calculating metrics

In [ ]:
all_tp = 0
all_fp = 0
all_tn = 0
all_fn = 0

Define a function to calculate the evaluation metrics (accuracy, recall, precision, and F1-score) for a given set of predictions and actual labels

In [1]:
def calculate_metrics(ground_truth, predictions):

    tp = sum(1 for gt, pred in zip(ground_truth, predictions) if gt == 1 and pred == 1)
    fp = sum(1 for gt, pred in zip(ground_truth, predictions) if gt == 0 and pred == 1)
    tn = sum(1 for gt, pred in zip(ground_truth, predictions) if gt == 0 and pred == 0)
    fn = sum(1 for gt, pred in zip(ground_truth, predictions) if gt == 1 and pred == 0)
    
    global all_tp, all_fp, all_tn, all_fn
    all_tp += tp
    all_fp += fp
    all_tn += tn 
    all_fn += fn

    metrics = {}
    metrics['accuracy'] = (tp + tn) / (tp + fp + tn + fn)
    metrics['recall'] = tp / (tp + fn)
    metrics['precision'] = tp / (tp + fp)
    metrics['F1'] = 2 * (metrics['precision'] * metrics['recall']) / (metrics['precision'] + metrics['recall'])
    return metrics